# Starter for MOA competition

In [ ]:
import pandas as pd
import numpy as np

inputPath = "../input/lish-moa/"

train_X = pd.read_csv(inputPath + "train_features.csv")
test_X = pd.read_csv(inputPath + "test_features.csv")
train_y = pd.read_csv(inputPath + "train_targets_scored.csv")

In [ ]:
train_X.head()

In [ ]:
train_y.head()

In [ ]:
test_X.head()

In [ ]:
test_X_id = test_X['sig_id']

In [ ]:
indexes = train_X.cp_type != 'ctl_vehicle'

In [ ]:
train_X = train_X.loc[indexes]

In [ ]:
train_y = train_y.loc[indexes]

# Preprocessing

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.plot(train_X['g-0'], '.')

In [ ]:
sns.distplot(train_X['g-0']);

In [ ]:
sns.distplot(train_X['c-0'])

In [ ]:
train_X[train_X['c-0'] == -10]

In [ ]:
train_X_ = train_X.drop(['sig_id', 'cp_type'], axis = 1)
test_X_ = test_X.drop(['sig_id', 'cp_type'], axis = 1)

In [ ]:
train_y_ = train_y.drop(['sig_id'], axis = 1)

In [ ]:
def lEncoder(value):
    if str(value) == 'D1': return 0
    return 1    

In [ ]:
train_X_['cp_dose'] = train_X_['cp_dose'].map(lEncoder)
test_X_['cp_dose'] = test_X_['cp_dose'].map(lEncoder)

In [ ]:
train_X_['cp_time'] = train_X_['cp_time'].map({24:-1, 48:0, 72:1})
test_X_['cp_time'] = test_X_['cp_time'].map({24:-1, 48:0, 72:1})

In [ ]:
train_X_.head()

In [ ]:
def encoder(value):
    ''' As per discussion forum'''
    if value >= 2:
        return 2
    elif value <= -2:
        return -2
    return value

In [ ]:
for col in train_X_.columns:
    if col != 'cp_time' or col != 'cp_dose':
        train_X_[col] = train_X_[col].map(encoder)
        test_X_[col] = train_X_[col].map(encoder)

In [ ]:
train_y_.head()

In [ ]:
X, y = train_X_, train_y_

In [ ]:
train_y.isnull().any().sum()

# Preparation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression

In [ ]:
y.iloc[:, 1].value_counts()

# Modelling

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 40)

In [ ]:
X_train.shape

In [ ]:
from tensorflow import keras

modelPred = keras.Sequential()

modelPred.add(keras.layers.Dense(128, input_shape = (874, )))
modelPred.add(keras.layers.BatchNormalization())
modelPred.add(keras.layers.Dense(512, activation = 'relu'))
modelPred.add(keras.layers.BatchNormalization())
modelPred.add(keras.layers.Dense(1024, activation = 'relu'))
modelPred.add(keras.layers.Dropout(0.5))
modelPred.add(keras.layers.Dense(512, activation = 'relu'))
modelPred.add(keras.layers.BatchNormalization())
modelPred.add(keras.layers.Dense(206, activation = 'sigmoid'))

modelPred.summary()

optim = keras.optimizers.Adam(learning_rate=1e-3)
modelPred.compile(optimizer = optim, loss = keras.losses.CategoricalCrossentropy())

In [ ]:
weights = {0:20000, 1:20}

history = modelPred.fit(
    X_train,
    y_train,
    batch_size = 32,
    epochs = 5,
    class_weight = weights
    #validation_data=(X_val, y_val)
)
'''
history = modelPred.fit(
    X_train,
    y_train,
    batch_size = 64,
    epochs = 2,
    class_weight = weights,
    validation_data=(X_val, y_val)
)''';

In [ ]:
y_predictions = modelPred.predict(test_X_)

y_predictions.shape

test_X_id.values.shape

final = np.hstack([test_X_id.values.reshape(-1, 1), y_predictions])

In [ ]:
modelPred.fit?

In [ ]:
final.shape

In [ ]:
final.shape

test_X.columns.shape

submission = pd.DataFrame(final, columns = train_y.columns)

submission.head()

submission.to_csv('submission.csv', index = False)